In [16]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn

In [17]:
df = pd.read_parquet("escooter_history_2022.parquet")

In [18]:
df

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
0,2020-01-04 00:00:09,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
1,2020-01-04 00:00:41,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
2,2020-01-04 00:01:20,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
3,2020-01-04 00:04:12,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
4,2020-01-04 00:15:19,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
...,...,...,...,...,...,...,...,...,...
3755410,2022-01-03 23:58:10,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755411,2022-01-03 23:58:28,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755412,2022-01-03 23:58:46,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755413,2022-01-03 23:59:16,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755415 entries, 0 to 3755414
Data columns (total 9 columns):
 #   Column               Dtype         
---  ------               -----         
 0   datetime             datetime64[ns]
 1   holiday              float64       
 2   workingday           float64       
 3   weather              object        
 4   temp                 float64       
 5   atemp                float64       
 6   humidity             float64       
 7   windspeed            float64       
 8   registered_customer  boolean       
dtypes: boolean(1), datetime64[ns](1), float64(6), object(1)
memory usage: 236.4+ MB


In [19]:
df.loc[df.temp > 50, "temp"] = df.temp.div(100)


In [20]:
bins = [-1,10,20,30,40,50,60,70,80,90,100]
df['temp'] = pd.cut(df['temp'], bins)
df["atemp"] = pd.cut(df["atemp"], bins)
df["humidity"] = pd.cut(df["humidity"], bins)
df["windspeed"] = pd.cut(df["windspeed"], bins)

In [21]:
df.dtypes

datetime               datetime64[ns]
holiday                       float64
workingday                    float64
weather                        object
temp                         category
atemp                        category
humidity                     category
windspeed                    category
registered_customer           boolean
dtype: object

In [24]:
df

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
0,2020-01-04 00:00:09,0.0,0.0,"clear, few clouds","(-1, 10]","(10, 20]","(80, 90]","(-1, 10]",True
1,2020-01-04 00:00:41,0.0,0.0,"clear, few clouds","(-1, 10]","(10, 20]","(80, 90]","(-1, 10]",True
2,2020-01-04 00:01:20,0.0,0.0,"clear, few clouds","(-1, 10]","(10, 20]","(80, 90]","(-1, 10]",True
3,2020-01-04 00:04:12,0.0,0.0,"clear, few clouds","(-1, 10]","(10, 20]","(80, 90]","(-1, 10]",True
4,2020-01-04 00:15:19,0.0,0.0,"clear, few clouds","(-1, 10]","(10, 20]","(80, 90]","(-1, 10]",True
...,...,...,...,...,...,...,...,...,...
3755410,2022-01-03 23:58:10,0.0,1.0,"clear, few clouds","(10, 20]","(10, 20]","(60, 70]","(-1, 10]",True
3755411,2022-01-03 23:58:28,0.0,1.0,"clear, few clouds","(10, 20]","(10, 20]","(60, 70]","(-1, 10]",True
3755412,2022-01-03 23:58:46,0.0,1.0,"clear, few clouds","(10, 20]","(10, 20]","(60, 70]","(-1, 10]",True
3755413,2022-01-03 23:59:16,0.0,1.0,"clear, few clouds","(10, 20]","(10, 20]","(60, 70]","(-1, 10]",True


In [22]:
nan_rows = df[df.isna().any(axis=1)]

In [23]:
nan_rows

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer


In [22]:
df_3 = df.groupby(pd.Grouper(key="datetime",freq="H")).agg({"registered_customer": "count","temp": "unique","humidity": "unique", "windspeed": "unique"}).reset_index()

In [23]:
df_3

,datetime,registered_customer,temp,humidity,windspeed
0,2020-01-04 00:00:00,16,"[(0, 10]] Categories (10, interval[int64, righ...","[(80, 90]] Categories (10, interval[int64, rig...","[NaN] Categories (10, interval[int64, right]):..."
1,2020-01-04 01:00:00,40,"[(0, 10]] Categories (10, interval[int64, righ...","[(70, 80]] Categories (10, interval[int64, rig...","[NaN] Categories (10, interval[int64, right]):..."
2,2020-01-04 02:00:00,32,"[(0, 10]] Categories (10, interval[int64, righ...","[(70, 80]] Categories (10, interval[int64, rig...","[NaN] Categories (10, interval[int64, right]):..."
3,2020-01-04 03:00:00,13,"[(0, 10]] Categories (10, interval[int64, righ...","[(70, 80]] Categories (10, interval[int64, rig...","[NaN] Categories (10, interval[int64, right]):..."
4,2020-01-04 04:00:00,1,"[(0, 10]] Categories (10, interval[int64, righ...","[(70, 80]] Categories (10, interval[int64, rig...","[NaN] Categories (10, interval[int64, right]):..."
...,...,...,...,...,...
17539,2022-01-03 19:00:00,371,"[(10, 20]] Categories (10, interval[int64, rig...","[(50, 60]] Categories (10, interval[int64, rig...","[(10, 20]] Categories (10, interval[int64, rig..."
17540,2022-01-03 20:00:00,282,"[(10, 20]] Categories (10, interval[int64, rig...","[(50, 60]] Categories (10, interval[int64, rig...","[(10, 20]] Categories (10, interval[int64, rig..."
17541,2022-01-03 21:00:00,245,"[(10, 20]] Categories (10, interval[int64, rig...","[(50, 60]] Categories (10, interval[int64, rig...","[(10, 20]] Categories (10, interval[int64, rig..."
17542,2022-01-03 22:00:00,179,"[(10, 20]] Categories (10, interval[int64, rig...","[(50, 60]] Categories (10, interval[int64, rig...","[(0, 10]] Categories (10, interval[int64, righ..."


In [26]:
px.scatter(df,x="datetime",y="temp")